In [1]:
from IPython.core.display import display, HTML
#makes the display bars longer and spread out 100% across the screen width
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

import collections

import warnings
warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

import db_queries as db
from get_draws.api import get_draws
import vivarium_helpers.id_helper as idh

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
import model_validation.vivarium_raw_output as vro

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/nathaniel/scratch__gitignore__
ndbs
Wed Aug 11 17:29:06 PDT 2021


In [6]:
import gbd_mapping

ModuleNotFoundError: No module named 'gbd_mapping'

In [8]:
idh.ids_to_names('rei', 339)

rei_id
339    Low birth weight and short gestation
Name: rei_name, dtype: object

In [7]:
LBWSG_REI_ID = idh.list_ids('rei', 'Low birth weight and short gestation')
LBWSG_REI_ID

339

In [11]:
GBD_2019_ROUND_ID = idh.list_ids('gbd_round', '2019')
GBD_2019_ROUND_ID

6

In [14]:
ETHIOPIA_ID = idh.list_ids('location', 'Ethiopia')
ETHIOPIA_ID

179

In [16]:
lbwsg_exposure = get_draws(
        'rei_id',
        gbd_id=LBWSG_REI_ID,
        source='exposure',
        location_id=ETHIOPIA_ID,
        year_id=2019,
#         sex_id=sex_ids, # Default is [1,2], but 3 also exists
        gbd_round_id=GBD_2019_ROUND_ID,
        status='best',
        decomp_step='step4',
    )
lbwsg_exposure

,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,parameter,measure_id,metric_id,draw_0,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,339,10788.0,179,2019,2,1,cat35,5,3,0.010837,...,0.010923,0.012334,0.010810,0.011132,0.011440,0.010278,0.011802,0.013016,0.010952,0.010837
1,339,10788.0,179,2019,3,1,cat35,5,3,0.011847,...,0.009794,0.011362,0.010923,0.010372,0.010661,0.008897,0.011417,0.011123,0.012206,0.011847
2,339,10788.0,179,2019,164,1,cat35,5,3,0.010191,...,0.010830,0.012246,0.014052,0.009975,0.010527,0.010849,0.010435,0.010666,0.010631,0.010191
3,339,10788.0,179,2019,2,2,cat35,5,3,0.010399,...,0.008424,0.009330,0.010667,0.006898,0.009976,0.009825,0.010281,0.008107,0.011710,0.010399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,339,NaN,179,2019,3,1,cat125,5,3,0.081695,...,0.036594,0.026499,0.087776,0.027117,0.000000,0.034552,0.053251,0.068818,0.077009,0.081695
351,339,NaN,179,2019,3,2,cat125,5,3,0.007514,...,0.099575,0.058358,0.000000,0.074324,0.016210,0.017585,0.087507,0.081632,0.005895,0.007514
352,339,NaN,179,2019,164,1,cat125,5,3,0.103147,...,0.039458,0.055518,0.039805,0.008661,0.050811,0.073197,0.000000,0.000000,0.000000,0.103147
353,339,NaN,179,2019,164,2,cat125,5,3,0.076497,...,0.100284,0.000000,0.000000,0.000000,0.000000,0.018877,0.012220,0.013844,0.000000,0.076497


In [26]:
isinstance(None, pd.DataFrame)

False

In [54]:
def get_draw_cols(draws_or_df=None):
    if isinstance(draws_or_df, pd.DataFrame):
        draw_cols = draws_or_df.columns.filter(regex=r'^draw_\d{1,3}$').columns.to_list()
    else:
        if draws_or_df is None:
            draws_or_df = range(1000) # draws
        draw_cols = [f'draw_{i}' for i in draws_or_df]
    return draw_cols

def get_draw_numbers(draw_cols_or_df=None):
    if isinstance(draw_cols_or_df, pd.DataFrame):
        draw_cols_or_df = draw_cols_or_df.columns.filter(regex=r'^draw_\d{1,3}$').columns.to_list()
    if draw_cols_or_df is not None:
        draws = [int(draw.replace('draw_', '')) for draw in draw_cols_or_df]
    else:
        draws = list(range(1000))
    return draws

#     if draw_cols is None:
#         return list(range(1000))
#     else:
#         return [int(draw.replace('draw_', '')) for draw in draw_cols]

draw_cols = get_draw_cols()
get_draw_numbers(draw_cols)[:20]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [42]:
help(get_draws)

Help on function get_draws in module get_draws.api:

get_draws(gbd_id_type, gbd_id, source, measure_id=None, location_id=None, year_id=None, age_group_id=None, sex_id=None, metric_id=None, version_id=None, status='best', gbd_round_id=None, num_workers=None, downsample=False, decomp_step=None, release_id=None, n_draws=None)
    Get draws for given arguments.
    
    Args:
        gbd_id_type(str or strlist): The gbd entity to retrieve draws for.
            Currently only 'cause_id', 'modelable_entity_id', 'rei_id',
            'sequela_id', and 'indicator_component_id are supported.
    
            Note -- gbd_id_type arguments must be paired 1 to 1 with
            gbd_id arguments if you are specifying multiple gbd_id_types.
            For example::
    
                get_draws(gbd_id_type=['cause_id', 'rei_id', rei_id],
                        gbd_id=[294, 82, 83], ... )
    
        gbd_id(int or intlist): The id(s) within the gbd_id_type(s) whose
            draws will be ret

In [21]:
cat_125 = lbwsg_exposure.query("parameter == 'cat125'")
cat_125

,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,parameter,measure_id,metric_id,draw_0,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
348,339,NaN,179,2019,2,1,cat125,5,3,0.045396,...,0.011591,0.000000,0.114671,0.031678,0.027457,0.041576,0.102746,0.025516,0.057051,0.045396
349,339,NaN,179,2019,2,2,cat125,5,3,0.080416,...,0.000000,0.000000,0.082397,0.037949,0.001475,0.000000,0.140591,0.029284,0.000000,0.080416
350,339,NaN,179,2019,3,1,cat125,5,3,0.081695,...,0.036594,0.026499,0.087776,0.027117,0.000000,0.034552,0.053251,0.068818,0.077009,0.081695
351,339,NaN,179,2019,3,2,cat125,5,3,0.007514,...,0.099575,0.058358,0.000000,0.074324,0.016210,0.017585,0.087507,0.081632,0.005895,0.007514
352,339,NaN,179,2019,164,1,cat125,5,3,0.103147,...,0.039458,0.055518,0.039805,0.008661,0.050811,0.073197,0.000000,0.000000,0.000000,0.103147
353,339,NaN,179,2019,164,2,cat125,5,3,0.076497,...,0.100284,0.000000,0.000000,0.000000,0.000000,0.018877,0.012220,0.013844,0.000000,0.076497


In [25]:
cat_125[draw_cols]

,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
348,0.045396,0.000000,0.042858,0.049351,0.045250,0.081505,0.084975,0.044852,0.076747,0.067884,...,0.011591,0.000000,0.114671,0.031678,0.027457,0.041576,0.102746,0.025516,0.057051,0.045396
349,0.080416,0.000000,0.046337,0.004334,0.094414,0.001382,0.142517,0.000000,0.052467,0.018983,...,0.000000,0.000000,0.082397,0.037949,0.001475,0.000000,0.140591,0.029284,0.000000,0.080416
350,0.081695,0.056268,0.009420,0.010716,0.051840,0.035180,0.070338,0.024013,0.048528,0.019805,...,0.036594,0.026499,0.087776,0.027117,0.000000,0.034552,0.053251,0.068818,0.077009,0.081695
351,0.007514,0.112071,0.031478,0.000000,0.030742,0.000000,0.069386,0.000000,0.108058,0.016421,...,0.099575,0.058358,0.000000,0.074324,0.016210,0.017585,0.087507,0.081632,0.005895,0.007514
352,0.103147,0.041516,0.085216,0.030378,0.092911,0.020212,0.018094,0.047914,0.042450,0.094292,...,0.039458,0.055518,0.039805,0.008661,0.050811,0.073197,0.000000,0.000000,0.000000,0.103147
353,0.076497,0.011444,0.030402,0.000000,0.000000,0.039862,0.118188,0.081143,0.001062,0.044093,...,0.100284,0.000000,0.000000,0.000000,0.000000,0.018877,0.012220,0.013844,0.000000,0.076497


# Idenify age groups and sexes in exposure data

In [30]:
ages = idh.ids_to_names('age_group', *lbwsg_exposure.age_group_id.unique())
ages

age_group_id
2      Early Neonatal
3       Late Neonatal
164             Birth
Name: age_group_name, dtype: object

In [32]:
sexes = idh.ids_to_names('sex', *lbwsg_exposure.sex_id.unique())
sexes

sex_id
1      Male
2    Female
Name: sex, dtype: object

In [37]:
age_ids = idh.names_to_ids('age_group', *ages)
age_ids

age_group_name
Early Neonatal      2
Late Neonatal       3
Birth             164
Name: age_group_id, dtype: int64

In [36]:
sex_ids = idh.names_to_ids('sex', *sexes)
sex_ids

sex
Male      1
Female    2
Name: sex_id, dtype: int64

# Split exposure data by age and sex

In [35]:
exposure_by_age_sex = {
    (age, sex): lbwsg_exposure.query(f"age_group_id == {age} and sex_id == {sex}") 
    for age in lbwsg_exposure.age_group_id.unique()
    for sex in lbwsg_exposure.sex_id.unique()
}
exposure_by_age_sex[(2,1)]

,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,parameter,measure_id,metric_id,draw_0,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,339,10788.0,179,2019,2,1,cat35,5,3,0.010837,...,0.010923,0.012334,0.010810,0.011132,0.011440,0.010278,0.011802,0.013016,0.010952,0.010837
6,339,10785.0,179,2019,2,1,cat32,5,3,0.014996,...,0.014963,0.013701,0.011945,0.013099,0.013423,0.014721,0.013641,0.013965,0.014369,0.014996
12,339,10787.0,179,2019,2,1,cat34,5,3,0.010429,...,0.009952,0.007347,0.007979,0.009153,0.008429,0.009225,0.008181,0.008592,0.009302,0.010429
18,339,10789.0,179,2019,2,1,cat36,5,3,0.004333,...,0.005635,0.004129,0.004518,0.004040,0.004910,0.005811,0.005076,0.005942,0.007631,0.004333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,339,10767.0,179,2019,2,1,cat14,5,3,0.000828,...,0.001092,0.001005,0.001347,0.001020,0.001033,0.001426,0.001167,0.001705,0.001086,0.000828
336,339,10761.0,179,2019,2,1,cat8,5,3,0.000010,...,0.000051,0.000039,0.000034,0.000174,0.000036,0.000036,0.000018,0.000042,0.000021,0.000010
342,339,10764.0,179,2019,2,1,cat11,5,3,0.000650,...,0.000779,0.000954,0.000563,0.000866,0.000776,0.000909,0.000810,0.000605,0.000800,0.000650
348,339,NaN,179,2019,2,1,cat125,5,3,0.045396,...,0.011591,0.000000,0.114671,0.031678,0.027457,0.041576,0.102746,0.025516,0.057051,0.045396


In [40]:
enn_males = exposure_by_age_sex[(age_ids['Early Neonatal'], sex_ids['Male'])]
enn_males

,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,parameter,measure_id,metric_id,draw_0,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
0,339,10788.0,179,2019,2,1,cat35,5,3,0.010837,...,0.010923,0.012334,0.010810,0.011132,0.011440,0.010278,0.011802,0.013016,0.010952,0.010837
6,339,10785.0,179,2019,2,1,cat32,5,3,0.014996,...,0.014963,0.013701,0.011945,0.013099,0.013423,0.014721,0.013641,0.013965,0.014369,0.014996
12,339,10787.0,179,2019,2,1,cat34,5,3,0.010429,...,0.009952,0.007347,0.007979,0.009153,0.008429,0.009225,0.008181,0.008592,0.009302,0.010429
18,339,10789.0,179,2019,2,1,cat36,5,3,0.004333,...,0.005635,0.004129,0.004518,0.004040,0.004910,0.005811,0.005076,0.005942,0.007631,0.004333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,339,10767.0,179,2019,2,1,cat14,5,3,0.000828,...,0.001092,0.001005,0.001347,0.001020,0.001033,0.001426,0.001167,0.001705,0.001086,0.000828
336,339,10761.0,179,2019,2,1,cat8,5,3,0.000010,...,0.000051,0.000039,0.000034,0.000174,0.000036,0.000036,0.000018,0.000042,0.000021,0.000010
342,339,10764.0,179,2019,2,1,cat11,5,3,0.000650,...,0.000779,0.000954,0.000563,0.000866,0.000776,0.000909,0.000810,0.000605,0.000800,0.000650
348,339,NaN,179,2019,2,1,cat125,5,3,0.045396,...,0.011591,0.000000,0.114671,0.031678,0.027457,0.041576,0.102746,0.025516,0.057051,0.045396


In [41]:
birth_males = exposure_by_age_sex[(age_ids['Birth'], sex_ids['Male'])]
birth_males

,rei_id,modelable_entity_id,location_id,year_id,age_group_id,sex_id,parameter,measure_id,metric_id,draw_0,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
2,339,10788.0,179,2019,164,1,cat35,5,3,0.010191,...,0.010830,0.012246,0.014052,0.009975,0.010527,0.010849,0.010435,0.010666,0.010631,0.010191
8,339,10785.0,179,2019,164,1,cat32,5,3,0.014973,...,0.012881,0.012994,0.014592,0.014796,0.012113,0.013491,0.013391,0.013446,0.014384,0.014973
14,339,10787.0,179,2019,164,1,cat34,5,3,0.007526,...,0.009600,0.006898,0.009394,0.008421,0.008341,0.007677,0.008662,0.008703,0.007416,0.007526
20,339,10789.0,179,2019,164,1,cat36,5,3,0.004051,...,0.004360,0.004632,0.004346,0.004399,0.004984,0.004756,0.004394,0.005681,0.005994,0.004051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,339,10767.0,179,2019,164,1,cat14,5,3,0.001302,...,0.001760,0.001053,0.001854,0.001875,0.001330,0.001652,0.002408,0.001417,0.001792,0.001302
338,339,10761.0,179,2019,164,1,cat8,5,3,0.001248,...,0.001266,0.001248,0.001262,0.001211,0.000967,0.001007,0.001073,0.001151,0.000979,0.001248
344,339,10764.0,179,2019,164,1,cat11,5,3,0.001591,...,0.001562,0.001196,0.001686,0.001545,0.001401,0.001351,0.001921,0.001829,0.001993,0.001591
352,339,NaN,179,2019,164,1,cat125,5,3,0.103147,...,0.039458,0.055518,0.039805,0.008661,0.050811,0.073197,0.000000,0.000000,0.000000,0.103147


# Check sums of prevalences by age, sex

In [44]:
prevalence_sums = lbwsg_exposure.groupby(['age_group_id', 'sex_id'])[draw_cols].sum()
prevalence_sums

draw_0    draw_1  draw_2    draw_3    draw_4    draw_5  \
age_group_id sex_id                                                           
2            1          1.0  1.015221     1.0  1.000000  1.000000  1.000000   
             2          1.0  1.039361     1.0  1.000000  1.000000  1.000000   
3            1          1.0  1.000000     1.0  1.000000  1.000000  1.000000   
             2          1.0  1.000000     1.0  1.028119  1.000000  1.035371   
164          1          1.0  1.000000     1.0  1.000000  1.000000  1.000000   
             2          1.0  1.000000     1.0  1.022168  1.006859  1.000000   

                     draw_6    draw_7  draw_8  draw_9  ...  draw_990  \
age_group_id sex_id                                    ...             
2            1          1.0  1.000000     1.0     1.0  ...  1.000000   
             2          1.0  1.034583     1.0     1.0  ...  1.012262   
3            1          1.0  1.000000     1.0     1.0  ...  1.000000   
             2          1.0  1.114459     1.0     1.0  ...  1.000000   
164          1          1.0  1.000000     1.0     1.0  ...  1.000000   
             2          1.0  1.000000     1.0     1.0  ...  1.000000   

                     draw_991  draw_992  draw_993  draw_994  draw_995  \
age_group_id sex_id                                                     
2            1       1.005779  1.000000  1.000000  1.000000  1.000000   
             2       1.006607  1.000000  1.000000  1.000000  1.115491   
3            1       1.000000  1.000000  1.000000  1.034509  1.000000   
             2       1.000000  1.001635  1.000000  1.000000  1.000000   
164          1       1.000000  1.000000  1.000000  1.000000  1.000000   
             2       1.107339  1.012738  1.008722  1.019068  1.000000   

                     draw_996  draw_997  draw_998  draw_999  
age_group_id sex_id                                          
2            1       1.000000  1.000000  1.000000       1.0  
             2       1.000000  1.000000  1.016560       1.0  
3            1       1.000000  1.000000  1.000000       1.0  
             2       1.000000  1.000000  1.000000       1.0  
164          1       1.003941  1.011409  1.028894       1.0  
             2       1.000000  1.000000  1.007318       1.0  

[6 rows x 1000 columns]

In [47]:
prevalence_sums.loc[(2,1), prevalence_sums.loc[(2,1)]>1]

draw_1      1.015221
draw_10     1.003690
draw_23     1.020329
draw_31     1.016332
              ...   
draw_977    1.005356
draw_978    1.003532
draw_987    1.010748
draw_991    1.005779
Name: (2, 1), Length: 127, dtype: float64

In [50]:
exposure_by_cat = lbwsg_exposure.set_index(['age_group_id', 'sex_id','parameter'])[draw_cols].sort_index()
exposure_by_cat

draw_0    draw_1    draw_2    draw_3  \
age_group_id sex_id parameter                                           
2            1      cat10      0.000174  0.000211  0.000097  0.000074   
                    cat106     0.001128  0.001423  0.000899  0.000870   
                    cat11      0.000650  0.000810  0.000410  0.001200   
                    cat116     0.001643  0.001821  0.001378  0.001522   
...                                 ...       ...       ...       ...   
164          2      cat89      0.001424  0.002042  0.001649  0.000920   
                    cat90      0.001282  0.000393  0.000900  0.000589   
                    cat95      0.004494  0.002790  0.003477  0.003474   
                    cat96      0.002274  0.001526  0.001401  0.001914   

                                 draw_4    draw_5    draw_6    draw_7  \
age_group_id sex_id parameter                                           
2            1      cat10      0.000223  0.000156  0.000067  0.000118   
                    cat106     0.001120  0.000893  0.000978  0.000849   
                    cat11      0.000699  0.000735  0.000682  0.000617   
                    cat116     0.001775  0.001524  0.001688  0.001653   
...                                 ...       ...       ...       ...   
164          2      cat89      0.000993  0.001588  0.001765  0.001690   
                    cat90      0.000818  0.000643  0.000487  0.001720   
                    cat95      0.003739  0.003930  0.004308  0.003816   
                    cat96      0.001900  0.001445  0.001581  0.001436   

                                 draw_8    draw_9  ...  draw_990  draw_991  \
age_group_id sex_id parameter                      ...                       
2            1      cat10      0.000206  0.000138  ...  0.000113  0.000124   
                    cat106     0.000969  0.001081  ...  0.000606  0.000915   
                    cat11      0.000473  0.000863  ...  0.000779  0.000954   
                    cat116     0.001529  0.001685  ...  0.001440  0.001357   
...                                 ...       ...  ...       ...       ...   
164          2      cat89      0.001892  0.001595  ...  0.001434  0.001318   
                    cat90      0.000561  0.000745  ...  0.000580  0.000894   
                    cat95      0.003509  0.002481  ...  0.004558  0.003705   
                    cat96      0.001990  0.002765  ...  0.002807  0.004247   

                               draw_992  draw_993  draw_994  draw_995  \
age_group_id sex_id parameter                                           
2            1      cat10      0.000141  0.000136  0.000058  0.000206   
                    cat106     0.001075  0.000518  0.001171  0.000887   
                    cat11      0.000563  0.000866  0.000776  0.000909   
                    cat116     0.001777  0.001777  0.001693  0.001880   
...                                 ...       ...       ...       ...   
164          2      cat89      0.000806  0.001426  0.001345  0.001273   
                    cat90      0.001230  0.000503  0.000949  0.000538   
                    cat95      0.002381  0.004679  0.005050  0.003426   
                    cat96      0.001425  0.002308  0.003620  0.001826   

                               draw_996  draw_997  draw_998  draw_999  
age_group_id sex_id parameter                                          
2            1      cat10      0.000276  0.000269  0.000103  0.000174  
                    cat106     0.001154  0.000858  0.000870  0.001128  
                    cat11      0.000810  0.000605  0.000800  0.000650  
                    cat116     0.002422  0.002055  0.002009  0.001643  
...                                 ...       ...       ...       ...  
164          2      cat89      0.000895  0.001751  0.001657  0.001424  
                    cat90      0.001614  0.000617  0.000434  0.001282  
                    cat95      0.004110  0.002717  0.002845  0.004494  
                    cat96      0.003025  0

In [51]:
exposure_by_cat.loc[(2,1, 'cat125')]

draw_0      0.045396
draw_1      0.000000
draw_2      0.042858
draw_3      0.049351
              ...   
draw_996    0.102746
draw_997    0.025516
draw_998    0.057051
draw_999    0.045396
Name: (2, 1, cat125), Length: 1000, dtype: float64

In [52]:
exposure_by_cat.loc[(2,1, 'cat125'), exposure_by_cat.loc[(2,1, 'cat125')]==0]

draw_1      0.0
draw_10     0.0
draw_23     0.0
draw_31     0.0
           ... 
draw_977    0.0
draw_978    0.0
draw_987    0.0
draw_991    0.0
Name: (2, 1, cat125), Length: 127, dtype: float64

In [53]:
idh.names_to_ids('age_group', 'Post Neonatal')

age_group_name
Post Neonatal    4
Name: age_group_id, dtype: int64